# To access the Mongo_db follow instructions:

To dump your mongo_db call this command on your terminal

 - mongodump --db database_name --collection collection_name (It will create a folder called dump in the folder active directory)

To import mongo_db file to mongodb you can use the following command on your terminal:

 - mongorestore --db database_name path_to_bson_file

In [1]:
# Database dependencies.
from sqlalchemy import create_engine

#Sqlite engine
engine = create_engine("sqlite:///USEnergy.sqlite", echo=True)
conn = engine.connect()
 
# Api dependencies
import requests 

#Json dependencies
from pprint import pprint

#API key
from DriversConfig.config import API_KEY #Save API key as API_KEY in a folder called DriversConfig

#DataFrame dependencies
import pandas as pd 

#Webscrapping 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

#Random crap
import time

#Settings for headless mode.
options = webdriver.ChromeOptions()
options.add_argument('headless')

#path to the driver and load the options.
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Group Project2 - ProjectEnergy/Sebastian/DriversConfig/chromedriver",chrome_options = options) #Dowload Selenium chromedriver and save the driver in the folder in the repo and update Absolute path.





2020-09-08 19:28:05,063 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-08 19:28:05,064 INFO sqlalchemy.engine.base.Engine ()
2020-09-08 19:28:05,064 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-08 19:28:05,065 INFO sqlalchemy.engine.base.Engine ()


In [2]:
#Lower 48 list.
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Prices 

- Data is very structured, SQL can be a good idea here

In [4]:
#API key from config file.
api_key = API_KEY

#Empty dictionary to store prices
price_df = pd.DataFrame(columns = states)

#for state in states.
for state in states:

    #query each state. ALL = average combined rates, A = Annual. --> we can get more specific and get Residential, bussiness, etc. 
    priceQuery = f"ELEC.PRICE.{state}-ALL.A"

    #Api url.
    url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

    #Api reqeust. 
    statePriceRequest = requests.get(url)

    #Pulls list of list of with the information of prices and yaer.
    yearPricesList = statePriceRequest.json()['series'][0]['data']
    
    #Parse through the list and build DataFrame.
    for priceYear in yearPricesList:

        price_df.loc[priceYear[0],state] = priceYear[1]

#Save to Sqlite 
price_df.to_sql('Average Energy Prices', conn, if_exists='replace')

In [22]:
#To view the sqlite 
prices = pd.read_sql_query("SELECT * from 'Average Energy Prices'", conn)
prices.set_index('index', inplace = True)

state = 'TX'

outputPrice = prices.loc[:,state].to_json(f'Output Files/{state} prices.json')

outputPrice

2020-09-08 19:58:21,657 INFO sqlalchemy.engine.base.Engine SELECT * from 'Average Energy Prices'
2020-09-08 19:58:21,657 INFO sqlalchemy.engine.base.Engine ()


# Energy Generation

- Data is very unstructured, we might have to use NoSQL (Mongo_dB)


<img src="images/2019 US Energy Distribution.png">

In [63]:
#list of the US Energy Sources by EIA.
energySources = {"Natural Gas" : "NG",
                "Coal" : "COW",
                "Petroleum Liquids" : "PEL", 
                "Petroleum Coke" : "PC", 
                "Other Gasses": "OOG",
                "Nuclear" : "NUC",
                "Hydroelectric" : 'HYC',
                "Wind" : "WND",
                'Wood and Wood Derived Fuels' : 'WWW',
                'Other Biomass' : 'WAS',
                'Geothermal' : 'GEO',
                'All Other Renewables' : "AOR",
                "All Solar": 'TSN',
                'Other' : 'OTH',
                'Utility Scale Thermal' : 'STH',
                'Hydroelectic Pumped Storage' : 'HPS'
                }

#For state in states
for state in states:

    #Empty Df to store data
    energySrouce_df = pd.DataFrame(columns = energySources.keys(), index = range(2001,2020))

    
    try:
        #For every statem loop each energy source.
        for energySourceKey, energySourceValue in zip(energySources.keys(), energySources.values()):
            
            #builds api query.
            priceQuery = f"ELEC.GEN.{energySourceValue}-{state}-99.A"
            
            #API url
            url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

            #Api reqeust. 
            energySourceRequest = requests.get(url, allow_redirects = False) 

            #Extracts the year-energy liss
            energySourceList = energySourceRequest.json()['series'][0]['data']

            for energy in energySourceList:
                energySrouce_df.loc[int(energy[0]), energySourceKey] = energy[1]

            energySrouce_df = energySrouce_df.fillna(0)

            #Save to Sqlite 
            energySrouce_df.to_sql(f'{state} Generation', conn, if_exists='replace')
    
    #if energy surce is not found.
    except KeyError as K:
        print(f'{K}. {energySourceKey} not found in {state}.')

        time.sleep(5)

print('Done!')

 (2005, 6386.79365, 41729.31343, 163.96753, 582.54246, 0.0, 9920.991, 1740.21896, 92.544, 740.34998, 0, 0, 0, 0, 0, 0, 0), (2006, 5358.4816, 40116.49023, 135.70123, 741.48433, 0.0, 12233.515, 1678.5983, 101.376, 773.8628, 0, 0, 0, 0, 0, 0, 0), (2007, 6488.75032, 40028.24955, 159.71219, 853.70149, 0.0, 12910.319, 1516.08286, 109.283, 785.07895, 0, 0, 0, 0, 0, 0, 0), (2008, 5243.58889, 41705.86048, 96.00967, 835.15012, 0.0, 12154.51, 1616.14196, 487.141, 775.04025, 0, 0, 0, 0, 0, 0, 0)  ... displaying 10 of 19 total bound parameter sets ...  (2018, 16798.65559, 33322.37949, 36.08083, 101.4305, 0.0, 10129.122, 2392.00955, 1637.658, 883.11537, 0, 0, 0, 0, 0, 0, 0), (2019, 21136.97903, 26348.94969, 46.54271, 89.68668, 0.0, 10030.305, 1992.70442, 1649.21371, 841.48595, 0, 0, 0, 0, 0, 0, 0))
2020-09-04 21:07:38,711 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-04 21:07:38,713 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-04 21:0

In [23]:
#To view the sqlite 
state = "TX"
stateGeneration = pd.read_sql_query(f"SELECT * from '{state} Generation'", conn) #State Initials to display table.

stateGeneration.set_index('index', inplace = True)

energyGenerationOutput = stateGeneration.to_json(f'Output Files/{state} Energy Generation.json')


2020-09-08 19:58:32,148 INFO sqlalchemy.engine.base.Engine SELECT * from 'TX Generation'
2020-09-08 19:58:32,149 INFO sqlalchemy.engine.base.Engine ()


# Energy Consumption by State

<img src="images/Energy Classification.png">


In [18]:
# Empty dataframe to store the data 
EndUseCosumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(3)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-electricity'] 

    for parseClass in parseClasses:

        yummySoup = soup.find_all("li", class_={parseClass})[0].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        EndUseCosumption_df.loc[typeEnergy,state] = marketShare

#Save to Sqlite 
EndUseCosumption_df.to_sql('End User Consumption', conn, if_exists='replace')

2020-09-04 20:38:38,824 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("End User Consumption")
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("End User Consumption")
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:38:38,828 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "End User Consumption" (
	"index" TEXT, 
	"AL" TEXT, 
	"AK" TEXT, 
	"AZ" TEXT, 
	"AR" TEXT, 
	"CA" TEXT, 
	"CO" TEXT, 
	"CT" TEXT, 
	"DC" TEXT, 
	"DE" TEXT, 
	"FL" TEXT, 
	"GA" TEXT, 
	"HI" TEXT, 
	"ID" TEXT, 
	"IL" TEXT, 
	"IN" TEXT, 
	"IA" TEXT, 
	"KS" TEXT, 
	"KY" TEXT, 
	"LA" TEXT, 
	"ME" TEXT, 
	"MD" TEXT, 
	"MA" TEXT, 
	"MI" TEXT, 
	"MN" TEXT, 
	"MS" TEXT, 
	"MO" TEXT, 
	"MT" TEXT, 
	"NE" TEXT, 
	"NV" TEXT, 
	"NH" TEXT, 
	"NJ" TEXT, 
	"NM" TEXT, 
	"NY" TEXT, 
	"NC" TEXT, 
	"ND" TEXT, 
	"OH" TEXT, 
	"OK" TEXT, 
	"OR" TEXT, 
	"PA" TEXT, 
	"RI" TEXT, 
	"SC" TEXT, 
	"SD" TEXT, 
	"TN

In [24]:
#To view the sqlite 
endUser = pd.read_sql_query("SELECT * from 'End User Consumption'", conn)
endUser

state = 'TX'

endUser.set_index('index', inplace= True)

endUser.loc[:,state].to_json(f'Output Files/{state} End User Consumption.json')

2020-09-08 19:58:45,803 INFO sqlalchemy.engine.base.Engine SELECT * from 'End User Consumption'
2020-09-08 19:58:45,805 INFO sqlalchemy.engine.base.Engine ()


In [19]:
#Empty dataframe to store the data 
ElectricPowerConsumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(4)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-nuclear'] 

    for parseClass in parseClasses:

        #From the soup, the electricity generation is in the index 1.
        indexNum = 1

        #Nuclear energy is only for electricity generation. It only has 1 index = 0
        if parseClass == 'info-nuclear' :
            indexNum = 0

        yummySoup = soup.find_all("li", class_={parseClass})[indexNum].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        ElectricPowerConsumption_df.loc[typeEnergy,state] = marketShare

#Save to Sqlite 
ElectricPowerConsumption_df.to_sql('Electric Power Consumption', conn, if_exists='replace')


2020-09-04 20:44:35,314 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Electric Power Consumption")
2020-09-04 20:44:35,314 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:44:35,315 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Electric Power Consumption")
2020-09-04 20:44:35,315 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:44:35,319 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Electric Power Consumption" (
	"index" TEXT, 
	"AL" TEXT, 
	"AK" TEXT, 
	"AZ" TEXT, 
	"AR" TEXT, 
	"CA" TEXT, 
	"CO" TEXT, 
	"CT" TEXT, 
	"DC" TEXT, 
	"DE" TEXT, 
	"FL" TEXT, 
	"GA" TEXT, 
	"HI" TEXT, 
	"ID" TEXT, 
	"IL" TEXT, 
	"IN" TEXT, 
	"IA" TEXT, 
	"KS" TEXT, 
	"KY" TEXT, 
	"LA" TEXT, 
	"ME" TEXT, 
	"MD" TEXT, 
	"MA" TEXT, 
	"MI" TEXT, 
	"MN" TEXT, 
	"MS" TEXT, 
	"MO" TEXT, 
	"MT" TEXT, 
	"NE" TEXT, 
	"NV" TEXT, 
	"NH" TEXT, 
	"NJ" TEXT, 
	"NM" TEXT, 
	"NY" TEXT, 
	"NC" TEXT, 
	"ND" TEXT, 
	"OH" TEXT, 
	"OK" TEXT, 
	"OR" TEXT, 
	"PA" TEXT, 
	"RI" TEXT, 
	"SC" TEXT, 

In [25]:
#To view the sqlite 
electricityGeneration = pd.read_sql_query("SELECT * from 'Electric Power Consumption'", conn)
electricityGeneration

state = 'TX'

electricityGeneration.set_index('index', inplace= True)

electricityGeneration.loc[:,state].to_json(f'Output Files/{state} Electricity Consumption.json')

2020-09-08 19:58:55,135 INFO sqlalchemy.engine.base.Engine SELECT * from 'Electric Power Consumption'
2020-09-08 19:58:55,136 INFO sqlalchemy.engine.base.Engine ()
